In [3]:
import pandas as pd
import numpy as np
from scipy.special import entr
from collections import defaultdict

In [8]:
trajs_combined_balanced = pd.read_pickle("../data/786/786_trajs_combined_balanced.p")
dfa = trajs_combined_balanced

In [17]:
### 이 코드에서 계산중에 같이 collect할 수 있을거라 생각했는데 , 포기.
from collections import defaultdict

def freq_seq_enum(sequences, min_support):
    '''Enumerates all frequent sequences.

       :param sequences: A sequence of sequences.
       :param min_support: The minimal support of a set to be included.
       :rtype: A set of (frequent_sequence, support).
    '''
    freq_seqs = set()
    _freq_seq(sequences, tuple(), 0, 0, min_support, freq_seqs)
    return freq_seqs


def _freq_seq(sdb, prefix, prefix_support, revisit_support, min_support, freq_seqs):
    if prefix:
        print('prefix: yes', prefix)
        freq_seqs.add((prefix, prefix_support, revisit_support))
        print('freq seqs', freq_seqs)
    locally_frequents = _local_freq_items(sdb, prefix, min_support)
    print('locally frequents', locally_frequents)
    if not locally_frequents:
        print ('not locally frequents')
        return
    for (item, support1, support2) in locally_frequents:
        new_prefix = prefix + (item,)
        new_sdb = _project(sdb, new_prefix)
        print('new_prefix', new_prefix)
        print('new_sdb', new_sdb)
        _freq_seq(new_sdb, new_prefix, support1, support2, min_support, freq_seqs)


def _local_freq_items(sdb, prefix, min_support):
    items = defaultdict(int)  ## for support
    items2 = defaultdict(int)  ## for revisit_intention_support
    freq_items = []
    for entry in sdb:
        visited = set()
        for element in entry[0]:
            print (element)
            if element not in visited:
                items[element] += 1
                items2[element] += entry[1]
                visited.add(element)
    # Sorted is optional. Just useful for debugging for now.
    for item in items:
        support = items[item] ## support
        support2 = items2[item] ## revisit_intention
        if support >= min_support:
            freq_items.append((item, support, support2))
    return freq_items


def _project(sdb, prefix):
    new_sdb = []
    if not prefix:
        return sdb
    current_prefix_item = prefix[-1]
    for entry in sdb:
        j = 0
        projection = None
        for item in entry[0]:
            if item == current_prefix_item:
                projection = (entry[0][j + 1:], entry[1])
                break
            j += 1
        if projection:
            new_sdb.append(projection)
    return new_sdb

In [12]:
seqs = dfa.apply(lambda x: (x['traj'], x['revisit_intention']), axis=1)

In [21]:
seqs2 = [('abab', 1.0), ('ab', 1.0)]

In [14]:
seqs.head(5)

16767_25e9517c47319ab64486403f94348dd6         ([out, in, 2f, 2f-inner, 1f], 0.0)
16955_53f0a6276486cc80405cf534d913fcbb    ([out, in, 1f, 1f-right, 1f-left], 0.0)
16734_5b37d580c76a8c8c245cad3418349acb                       ([out, in, 1f], 0.0)
16736_70968e4fb8894b74997f6f4e89b32bee                       ([out, in, 1f], 1.0)
16847_e0b87d079ff087efd100c423d88d0fbd                       ([out, in, 1f], 0.0)
dtype: object

In [22]:
freq_seq_enum(seqs2, 1)

a
b
a
b
a
b
locally frequents [('b', 2, 2.0), ('a', 2, 2.0)]
new_prefix ('b',)
new_sdb [('ab', 1.0), ('', 1.0)]
prefix: yes ('b',)
freq seqs {(('b',), 2, 2.0)}
a
b
locally frequents [('b', 1, 1.0), ('a', 1, 1.0)]
new_prefix ('b', 'b')
new_sdb [('', 1.0)]
prefix: yes ('b', 'b')
freq seqs {(('b',), 2, 2.0), (('b', 'b'), 1, 1.0)}
locally frequents []
not locally frequents
new_prefix ('b', 'a')
new_sdb [('b', 1.0)]
prefix: yes ('b', 'a')
freq seqs {(('b', 'a'), 1, 1.0), (('b',), 2, 2.0), (('b', 'b'), 1, 1.0)}
b
locally frequents [('b', 1, 1.0)]
new_prefix ('b', 'a', 'b')
new_sdb [('', 1.0)]
prefix: yes ('b', 'a', 'b')
freq seqs {(('b', 'a'), 1, 1.0), (('b',), 2, 2.0), (('b', 'b'), 1, 1.0), (('b', 'a', 'b'), 1, 1.0)}
locally frequents []
not locally frequents
new_prefix ('a',)
new_sdb [('bab', 1.0), ('b', 1.0)]
prefix: yes ('a',)
freq seqs {(('b', 'a'), 1, 1.0), (('a',), 2, 2.0), (('b',), 2, 2.0), (('b', 'b'), 1, 1.0), (('b', 'a', 'b'), 1, 1.0)}
b
a
b
b
locally frequents [('b', 2, 2.0), ('a

{(('a',), 2, 2.0),
 (('a', 'a'), 1, 1.0),
 (('a', 'a', 'b'), 1, 1.0),
 (('a', 'b'), 2, 2.0),
 (('a', 'b', 'a'), 1, 1.0),
 (('a', 'b', 'a', 'b'), 1, 1.0),
 (('a', 'b', 'b'), 1, 1.0),
 (('b',), 2, 2.0),
 (('b', 'a'), 1, 1.0),
 (('b', 'a', 'b'), 1, 1.0),
 (('b', 'b'), 1, 1.0)}